In [1]:
import tensorflow as tf
from keras.applications import ResNet50, DenseNet121
from keras.layers import Conv2D, BatchNormalization
import numpy as np

Using TensorFlow backend.


In [2]:
from vbranch.applications.resnet import ResNet50 as vb_ResNet50

In [3]:
with tf.variable_scope('keras_model'):
    keras_model = ResNet50(weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
len(keras_model.layers)

177

In [6]:
with tf.variable_scope('vb_model'):
    inputs = tf.placeholder('float32', [None, 224, 224, 3])
    vb_model, assign_ops = vb_ResNet50(inputs, 1000, weights='imagenet')

Loading weights for ResNet50...


In [7]:
vb_model.summary()

i    Layer name                      Output shape       Parameters              Num param  Inbound                 
-------------------------------------------------------------------------------------------------------------------
     Input                           [None,224,224,3]                                                              
-------------------------------------------------------------------------------------------------------------------
0    conv1 (Conv2D)                  [None,112,112,64]  [7,7,3,64] [64]         9472       vb_model/Placeholder:0  
-------------------------------------------------------------------------------------------------------------------
1    bn_conv1 (BatchNormalization)   [None,112,112,64]  [64] [64]               128        conv1                   
-------------------------------------------------------------------------------------------------------------------
2    relu (Activation)               [None,112,112,64]                  

In [8]:
len(vb_model.layers)

174

In [9]:
# layer1 = keras_model.layers[2].get_weights()

In [10]:
# assign_ops

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    vb_layer1 = vb_model.layers[-1].get_weights(sess=sess)
    sess.run(assign_ops)
    vb_layer1_assign = vb_model.layers[-1].get_weights(sess=sess)
#     print(sess.run(a))
#     sess.run(op)
#     print(sess.run(a))

In [12]:
# layer1[0]

In [13]:
vb_layer1

[array([[ 0.04326808, -0.04389165, -0.00125558, ..., -0.02989461,
         -0.02853836, -0.00826449],
        [ 0.03842824, -0.01835495, -0.02856161, ..., -0.02058643,
         -0.0035076 ,  0.03209823],
        [-0.03733534, -0.0314411 ,  0.03195931, ...,  0.01563994,
         -0.00127824, -0.03097734],
        ...,
        [ 0.01690513, -0.01143346,  0.02085436, ...,  0.03274724,
          0.0082619 , -0.01760569],
        [ 0.03517387,  0.04383385,  0.01996018, ..., -0.01880384,
         -0.02079274, -0.00625505],
        [ 0.02981445,  0.02670447, -0.03074491, ..., -0.00017582,
         -0.01415536, -0.04277418]], dtype=float32),
 array([ 1.83035396e-02, -3.13642323e-02, -1.35002509e-02, -1.75282136e-02,
        -3.00245509e-02,  1.89992674e-02,  4.12675366e-03, -4.46760207e-02,
        -4.91760150e-02, -5.28054312e-03,  2.57106498e-03,  4.08244506e-03,
         2.27432437e-02, -4.33563814e-03, -4.68288437e-02,  6.63252547e-03,
        -5.14198169e-02, -2.89132148e-02, -2.97334846e

In [14]:
vb_layer1_assign

[array([[ 0.04326808, -0.04389165, -0.00125558, ..., -0.02989461,
         -0.02853836, -0.00826449],
        [ 0.03842824, -0.01835495, -0.02856161, ..., -0.02058643,
         -0.0035076 ,  0.03209823],
        [-0.03733534, -0.0314411 ,  0.03195931, ...,  0.01563994,
         -0.00127824, -0.03097734],
        ...,
        [ 0.01690513, -0.01143346,  0.02085436, ...,  0.03274724,
          0.0082619 , -0.01760569],
        [ 0.03517387,  0.04383385,  0.01996018, ..., -0.01880384,
         -0.02079274, -0.00625505],
        [ 0.02981445,  0.02670447, -0.03074491, ..., -0.00017582,
         -0.01415536, -0.04277418]], dtype=float32),
 array([ 1.83035396e-02, -3.13642323e-02, -1.35002509e-02, -1.75282136e-02,
        -3.00245509e-02,  1.89992674e-02,  4.12675366e-03, -4.46760207e-02,
        -4.91760150e-02, -5.28054312e-03,  2.57106498e-03,  4.08244506e-03,
         2.27432437e-02, -4.33563814e-03, -4.68288437e-02,  6.63252547e-03,
        -5.14198169e-02, -2.89132148e-02, -2.97334846e

In [15]:
keras_model.layers[-1].get_weights()

[array([[-0.01490746,  0.0113374 , -0.05073728, ..., -0.02179668,
         -0.07764222,  0.01018347],
        [-0.00294467,  0.00319835,  0.01953556, ...,  0.03623696,
          0.00350259, -0.03321117],
        [-0.01751374,  0.00807406,  0.00851311, ..., -0.03024036,
          0.05494978, -0.02511911],
        ...,
        [ 0.025289  ,  0.0630148 ,  0.02041481, ..., -0.00508354,
         -0.03542514, -0.01306196],
        [-0.00623157, -0.01624131, -0.01221174, ...,  0.01376359,
          0.04087579, -0.0185826 ],
        [-0.02668471,  0.0130982 , -0.01847764, ...,  0.06304929,
          0.02561448, -0.00693536]], dtype=float32),
 array([-8.06632824e-03, -2.82376772e-03, -6.42026169e-03, -1.45175382e-02,
         1.85494672e-03, -1.18305106e-02,  4.32976376e-04,  6.83434773e-04,
        -4.48546931e-03, -5.43952826e-03, -7.74441753e-03, -4.83709993e-03,
        -1.29229026e-02, -3.47052747e-03, -2.76576495e-03, -6.09770464e-03,
        -1.26926170e-03, -6.55239029e-03,  7.65493792e

In [7]:
a = tf.get_variable('a', shape=[])
op = tf.assign(a, a+1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a))
    sess.run(op)
    print(sess.run(a))

-0.25432503
0.74567497


## Save Weights

In [7]:
keras_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='keras_model')
vb_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='vb_model')

In [8]:
def get_tensor_by_name(collection, name):
    for c in collection:
        if c.name == name:
            return c
    return

In [9]:
assign_ops = []

for w in vb_weights:
    name = w.name[len('vb_model/'):]
    # Remove suffix
    name = name[:name.index('/')]
    
    if name[:3] == 'res' or name[:4] == 'conv':
        # Conv
        ref_filter = get_tensor_by_name(vb_weights, 'vb_model/' + name + '/filter:0')
        ref_bias = get_tensor_by_name(vb_weights, 'vb_model/' + name + '/bias:0')
        value_filter = get_tensor_by_name(keras_weights, 'keras_model/' + name + '/kernel:0')
        value_bias = get_tensor_by_name(keras_weights, 'keras_model/' + name + '/bias:0')
        
        assign_ops.append(tf.assign(ref_filter, value_filter))
        assign_ops.append(tf.assign(ref_bias, value_bias))
    elif name[:2] == 'bn':
        # Batch norm
        ref_gamma = get_tensor_by_name(vb_weights, 'vb_model/' + name + '/scale:0')
        ref_beta = get_tensor_by_name(vb_weights, 'vb_model/' + name + '/beta:0')
        value_gamma = get_tensor_by_name(keras_weights, 'keras_model/' + name + '/gamma:0')
        value_beta = get_tensor_by_name(keras_weights, 'keras_model/' + name + '/beta:0')
        
        assign_ops.append(tf.assign(ref_gamma, value_gamma))
        assign_ops.append(tf.assign(ref_beta, value_beta))

In [10]:
# assign_ops

In [7]:
imagenet_weights = {}

for layer in keras_model.layers:
    name = layer.name
    
    if isinstance(layer, Conv2D):
        # Conv
        imagenet_weights[name] = {}
        weights = layer.get_weights()
        f = weights[0]
        assert len(f.shape) == 4
        b = weights[1]
        assert len(b.shape) == 1
        imagenet_weights[name] = {'filter' : f, 'bias': b}
    elif isinstance(layer, BatchNormalization):
        # Batch norm
        imagenet_weights[name] = {}
        weights = layer.get_weights()
        gamma = weights[0]
        beta = weights[1]
        imagenet_weights[name] = {'scale' : gamma, 'beta':beta}

In [8]:
import pickle
with open('weights/resnet50.pickle', 'wb') as pickle_out:
    pickle.dump(imagenet_weights, pickle_out)